In [2]:
#load data
import pandas as pd
def load_data(csv_file, sample_fraction=1.0):
    """Loads Question Pairs from a CSV file

    Args:
        csv_file (str): Path to csv_file
        sample_fraction (float): Fraction of data to sample, default is 1.0
    Returns:
        tuple: A tuple containing supervised data pairs
        returns [],[] on error
    """
    sentences1 = []
    sentences2 = []
    is_duplicate = []
    
    try:
        df = pd.read_csv(csv_file, encoding='utf-8')
        #print("Column names:", df.columns)
        
        if sample_fraction < 1.0:
            df = df.sample(frac=sample_fraction, random_state=42).reset_index(drop=True)
        
        sentence1_series = df['question1']
        sentence2_series = df['question2']
        is_duplicate_series = df['is_duplicate']
        
        sentences1 = sentence1_series.tolist()
        sentences2 = sentence2_series.tolist()
        is_duplicate = is_duplicate_series.tolist()
        
        if len(sentences1) != len(sentences2):
            raise ValueError("The number of sentences in question1 and question2 do not match.")
        else:
            print(f"Loaded {len(sentences1)} sentences.")
        return sentences1, sentences2, is_duplicate
        
    except FileNotFoundError:
        print("Wrong Path")
        return [],[],[]
    
    except Exception as e:
        print(f"An {e} Error Occurred")
        return [],[],[]

DATA_PATH = r'C:/Users/Jash\Documents/Research\Semantic Equivilance\SemanticEquivilance/question_pairs/questions.csv'
sentences1, sentences2, value = load_data(DATA_PATH, sample_fraction=0.001)


Loaded 404 sentences.


In [ ]:
import pennylane as qml
import numpy as np
from lambeq import AtomicType, BobcatParser, IQPAnsatz, Rewriter
from lambeq.backend.pennylane import to_pennylane as to_qml
import random
import torch

def get_circuit_state_vector(qml_qnode_func, num_wires):
    """
    Executes a PennyLane QNode function and returns the state vector.
    The qml_qnode_func should return a qml.QNode object.
    """
    dev = qml.device("default.qubit", wires=num_wires)

    @qml.qnode(dev)
    def state_vector_wrapper():
        qml_qnode_func() # Execute the actual circuit operations
        return qml.state()

    return state_vector_wrapper()

def swap_test(state1_vec, state2_vec):
    """
    Performs a Quantum Swap Test between two quantum state vectors.

    Args:
        state1_vec (np.ndarray): The first state vector.
        state2_vec (np.ndarray): The second state vector.

    Returns:
        float: The estimated squared overlap (fidelity) between the two states.
    """
    num_qubits = int(np.log2(len(state1_vec)))
    if 2**num_qubits != len(state1_vec):
        raise ValueError("State vectors must have a length that is a power of 2.")

    total_qubits = 1 + 2 * num_qubits #1 Ancilla qubit + 2 state qubits

    dev = qml.device("default.qubit", wires=total_qubits, shots=1000) # N = 1000 runs of the circuit for statistical significance

    @qml.qnode(dev)
    def circuit():
        # Step 1: Prepare the ancilla qubit in a superposition
        qml.Hadamard(wires=0)
        
        # Step 2: Prepare the two input states
        qml.StatePrep(state1_vec, wires=range(1, 1 + num_qubits))
        qml.StatePrep(state2_vec, wires=range(1 + num_qubits, 1 + 2 * num_qubits))

        # Step 3: Apply controlled-SWAP gates
        for i in range(num_qubits):
            qml.CSWAP(wires=[0, 1 + i, 1 + num_qubits + i]) #selects every register of phi and psi for swap

        # Step 4: Apply Hadamard to ancilla
        qml.Hadamard(wires=0)
        # Step 5: Measure the ancilla qubit
        return qml.sample(wires=0)

    measurement_results = circuit()
    prob_0 = np.sum(measurement_results == 0) / len(measurement_results)
    squared_overlap = 2 * prob_0 - 1

    return squared_overlap

def lambeq_sentence_to_state_vector(sentence, ansatz, parser, include_debug_prints=False):
    """
    Converts a natural language sentence into a quantum state vector
    using Lambeq's BobcatParser and IQPAnsatz, handling parameterization
    via PennyLaneModel.

    Args:
        sentence (str): The input sentence.
        ansatz (lambeq.ansatz.Ansatz): The quantum ansatz to apply.
        parser (lambeq.parser.Parser): The parser to convert sentence to diagram.

    Returns:
        tuple: A tuple containing the state vector (np.ndarray) and
               the number of qubits (int).
    """
    try:
        
        diagram = parser.sentence2diagram(sentence)
        rewritten_diagram = rewriter(diagram) # type: ignore[reportArgumentType]
        normalized_diagram = rewritten_diagram.normal_form() # Normalize the diagram
        if include_debug_prints:
            print(f"\n--- Debugging: Sentence '{sentence}' ---")
            #print(f"DisCoPy Diagram (before ansatz):\n{diagram}")

        circuit = ansatz(normalized_diagram)
        #if include_debug_prints:
            #print(f"DisCoPy Circuit (after ansatz):\n{circuit}")
            #circuit.draw(figsize=(20, 10), fontsize=6, textpad=0.3)#, draw_type_labels=False, fontsize=8,foliated=True) # Uncomment to visualize the circuit

        free_symbols = list(circuit.free_symbols)
        if include_debug_prints:
            #print(f"Free symbols in circuit: {free_symbols}")
            #print(f"Number of free symbols: {len(free_symbols)}")
            if not free_symbols:
                print(f"Warning: No free symbols found in circuit for sentence '{sentence}'. This means the circuit is not parametric.")
    
        #if include_debug_prints:
            #print("--- Step 3.1: Attempting to get number of qubits (circuit.n_qubits) ---")

        num_qubits = num_qubits = len(circuit.cod) if circuit.cod else 0 #number of qubits is determined by the codomain of the circuit          
        
        #if include_debug_prints:
            #print(f"--- Step 3.2: Successfully retrieved num_qubits: {num_qubits} ---")
            
        random.seed(42)  # Set a seed for reproducibility
        param_map = {}
        if free_symbols:
            random_params = [random.uniform(0, 2*np.pi) for _ in range(len(free_symbols))]
            param_map = {s: val for s, val in zip(free_symbols, random_params)}
            # if include_debug_prints:
            #     #print(f"Randomized Parameters map: {param_map}:")
            #     #for symbol, value in sorted(param_map.keys(), key=str):
            #      #   print(f"  {symbol}: {value:.4f}") # Print with formatting for readability
            #         print("--- Step 5: Preparing to substitute parameters into circuit (.lambd) ---")
            #         print(f"Type of free_symbols elements: {type(free_symbols[0]) if free_symbols else 'N/A'}")
            #         print(f"Type of param_map keys: {type(next(iter(param_map.keys()))) if param_map else 'N/A'}")
            #         print(f"First 3 param_map items: {list(param_map.items())[:3]}")
        else:
            if include_debug_prints:
                print("No free symbols found, using static circuit without parameterization.")
                
        #if include_debug_prints:
            #print("--- Step 6: Defining PennyLane device and QNode ---")
        dev = qml.device("default.qubit", wires=num_qubits)
        
        @qml.qnode(dev)
        def get_quantum_state_vector(circuit):
            
            temp_circuit = to_qml(circuit)
            temp_circuit.initialise_concrete_params(param_map) # Initialize the circuit with the parameter map
            temp_circuit.make_circuit()
            
            temp_circuit.eval() # Convert DisCoPy circuit to PennyLane circuit with parameter mapping
            state = qml.state() # Get the state vector
            if include_debug_prints:
                print("--- Step 7.5: Successfully converted DisCoPy circuit to PennyLane circuit ---")
                print(state)
            return temp_circuit
        
        #if include_debug_prints:
            #print("--- Step 7: Executing QNode to get state vector ---")
        state_vector = get_quantum_state_vector(circuit)
        if include_debug_prints:
            print("--- Step 8: Successfully generated state vector ---")
        return state_vector, num_qubits

    except Exception as e:
            print(f"CRITICAL ERROR: Failed to process circuit {e}")
            print("Returning a default normalized |0> state to allow program continuation.")
            default_state = np.array([1.+0.j, 0.+0.j]) #|0>
            default_state[0] = 1.0 # then set the first element to 1 to represent |0>
            return default_state, 1 #1 is default number of qubits for |0> state
        

In [131]:
if __name__ == "__main__":
    parser = BobcatParser()
    rewriter = Rewriter(['curry', 'prepositional_phrase', 'determiner'])
    ansatz = IQPAnsatz({AtomicType.NOUN: 3, AtomicType.SENTENCE: 1}, n_layers=3) #sentence number determines number of output qubits

    print("--- Generating states from sentences ---")

    sentence1 = "Alice loves the dog that Bob purchased."
    sentence2 = "Bob loves the dog that Alice sold."
    sentence3 = "The big cat sleeps peacefully."
    sentence4 = "The small bird sings loudly."

    sentences = [sentence1, sentence2, sentence3, sentence4]
    state_data = {}

    for s_idx, sentence in enumerate(sentences):
        try:
            state_vec, num_qubits = lambeq_sentence_to_state_vector(sentence, ansatz, parser, include_debug_prints=True)
            state_data[sentence] = (state_vec, num_qubits)
            print(f"Sentence {s_idx+1}: '{sentence}'")
            #if state_vec is not None:
                #print(f"State vector shape: {state_vec.shape}")
            #else:
                #print("State vector is None, indicating a failure to generate a valid state.")
        except Exception as e:
            print(f"Error processing sentence '{sentence}': {e}")
            state_data[sentence] = (None, None) #Failed

    print("\n--- Performing Swap Tests ---")
    
    # Filter out sentences that did not produce valid states

    valid_sentences = [s for s in sentences if state_data[s][0] is not None and state_data[s][1] is not None and state_data[s][1] > 0]

    if not valid_sentences:
        print("No valid multi-qubit states generated. Cannot perform Swap Tests meaningfully.")
    else:
        first_num_qubits = state_data[valid_sentences[0]][1]
        all_same_qubits = all(state_data[s][1] == first_num_qubits for s in valid_sentences)

        if not all_same_qubits:
            print("\nWarning: Not all valid sentences resulted in circuits with the same number of qubits.")
            print("Swap Test requires states to have the same number of qubits.")
            print("Pairs with different qubit counts will be skipped.")
            for s in valid_sentences:
                print(f"  '{s}': {state_data[s][1]} qubits")

        for i in range(len(valid_sentences)):
            for j in range(i, len(valid_sentences)):
                s1 = valid_sentences[i]
                s2 = valid_sentences[j]

                vec1, nq1 = state_data[s1]
                vec2, nq2 = state_data[s2]

                if nq1 == nq2:
                    print(f"\nSwap Test between '{s1}' and '{s2}':")
                    overlap = swap_test(vec1, vec2)
                    print(f"Estimated squared overlap: {overlap:.4f}")
                    if s1 == s2:
                        print(" (Expected to be close to 1.0 for identical states)")
                else:
                    print(f"\nSkipping Swap Test between '{s1}' ({nq1} qubits) and '{s2}' ({nq2} qubits) due to different qubit counts.")

--- Generating states from sentences ---

--- Debugging: Sentence 'Alice loves the dog that Bob purchased.' ---
--- Step 7.5: Successfully converted DisCoPy circuit to PennyLane circuit ---
state(wires=[])
CRITICAL ERROR: Failed to process circuit A quantum function must return either a single measurement, or a nonempty sequence of measurements.
Returning a default normalized |0> state to allow program continuation.
Sentence 1: 'Alice loves the dog that Bob purchased.'

--- Debugging: Sentence 'Bob loves the dog that Alice sold.' ---
--- Step 7.5: Successfully converted DisCoPy circuit to PennyLane circuit ---
state(wires=[])
CRITICAL ERROR: Failed to process circuit A quantum function must return either a single measurement, or a nonempty sequence of measurements.
Returning a default normalized |0> state to allow program continuation.
Sentence 2: 'Bob loves the dog that Alice sold.'

--- Debugging: Sentence 'The big cat sleeps peacefully.' ---
--- Step 7.5: Successfully converted DisC